# Projective Geometry

## Geometry
What is Geometry?

Well Geometry deals with the properties of figures in the plane or in space. \\There are lots of classification methods to break geometry into different parts. 

One useful distinction is between _Synthetic_ geometry and _Analytic_ geometry.
       

## Synthetic Geometry


Synthetic is what most of us think of as Geometry. i.e. drawing lines, and shapes and looking at points of intersection or angles etc. 

Indeed if you look at Euclid's Elements, all proofs are carried out using straight lines that we don't know a specific length of, or circles that we don't know the specific radius, but nonetheless, every figure can be built from the picture alone.

This idea of building up from simple parts using logic and reasoning is what is meant by synthesis, hence Synthetic Geometry.
![](images/Proof_of_Butterfly_theorem.png)
	
	
 

       


## Analytical Geometry
Analytical Geometry is based on the introduction of numerical coordinates and uses the techniques of algebra.
By doing this, Geometry, Analysis and Algebra were all brought together into one system.

To begin with, we normally always refer the algebra back to some picture or figure as if to prove that it is not just some sort of mathematical magic. 

But by using algebra we can free ourselves of the drawings (which, try as we might, can never really be exact) and move purely to abstract algebra to find solutions. 

In much the same way that imaginary numbers allowed us to find solutions to problems that could not appear on a graph, Analytic Geometry allows us to use all the systems of Algebra to produce a final result that, should we need to, we can always show pictorially.

The trouble for the student is that you can _see_ synthetic geometry but you can't see or interpret each of the steps in algebra. 

**If you can get over this, it will really pay you dividends.**
       

Theorems of elementary geometry are based on lengths (magnitudes), measures of angles, and areas/volumes. 

As we said in earlier lectures, there are geometrical transformations that can take place on figures that, while changing the figure, will retain some properties that are common to both the original and transformed figure.

Describing the properties that are retained in an algebraic fashion becomes useful for determining how a figure will change due to a transformation.
       

## Congruence

If two figures are congruent then they contain the same magnitudes and angles but not position. 
This means that the figure has undergone a rigid motion, (rotation, translation). 

Previously, we came up with algebraic formulations for these rigid body motions (translation, rotation) and we showed that they retained magnitudes (scalar product) and angles (cross product).
  

## Projective Transformation

In our list of geometrical tranformations, the one that preserved the least was the projective transform. 
There we stated only that it preserved straight lines. 

This may sound very limiting but ensuring that the straight lines in a figure are preserved under a transformation allows an enormous amount to be determined. 

And this is very important in Computer Vision because figures in the real world undergo a projective transform when being captured by a camera. 

So this class of transforms and the preservation of straight lines becomes a primary focus.

The study of projective geometry was considered long before the introduction of the camera. 

Artists such as da Vinci took it into account in order to draw realistic images, i.e. the real world is projected onto the canvas by the artist. 

The eye of the painter is the center of the projection. 

So it is drawn from the _perspective_ of the painter. 

Therefore the resulting image can be considered a _perspective_.

Lengths and angles are distorted, but in a way that is dependent on their relative positions.

As the human eye is used to seeing the world from a perspective view then the geometrical structure in an image can usually be recognised.
       

## Central Projection.

If we have two planes $\pi$ and $\pi'$ in a space and these planes do not have to be parallel to eachother. 

We can perform a central projection of $\pi$ onto $\pi'$ from a given center $O$ not lying in $\pi$ or $\pi'$ by defining the image of each point $P$ on $\pi$  to be that point $P'$ of $\pi'$, such that $P$ and $P'$ lie on the same straight line through $O$. 

![](images/ProjectionFromAPoint.png)
	

 
## Parallel Projection.


We may also perform a parallel projection. 
At the start we always do this, we show the concept for a central projection and then mention that there is a parallel projection as well, which we seem less interested in, but we will see later that when we move to algebra this can all be unified and suddenly life gets easier. 
In effect there are no exceptions that we have to worry about.
![](images/ParallelProjection.png)
	
	

## Projective Vs Metric Geometry

Any mapping of one figure onto another by a central or parallel projection, or by a succession of them is called a projective transformation.

We will call the body of geometrical propositions which are unaffected by any arbitrary projective transformations of figures he projective geometry of the plane or the line.

The Rigid body motions which preserve magnitudes and angles we will refer to as metric geometry (i.e. we can measure).
Some properties of projections are immediately obvious.

A point projects to a point. A Straight line projects to a straight line (hence preserves straight lines).
       
If a point $A$ and a straight line $l$ are incident (the point is on the line or the line goes through a point) then following the projective transformation the projected point $A'$ and the projected line $l'$ will also be incident.

So incidence of a point and a line is preserved under a projective projection. 

As we can already see, the preservation of straight lines is leading us to other interesting properties that are preserved.
We can go further. 
If three or more points are collinear, i.e. incident with some straight line, then their images will also be collinear. 
Be careful to realise, that the line itself, does not have to be visible to you for this to still be true. 

What about this, if three or more straight lines in the plane are concurrent (incident with some point) then their projections will also be concurrent straight lines.

What about two in both of the cases above?

While incidence, collinearity, and concurrence are projective properties that are preserved, the measures of the length and angle, and the ratios of magnitudes, are generally altered by projections.

Triangles will map to triangles but an equilateral or Isosceles is unlikely to project to and equilateral or Isosceles triangle after the projection.

## Parallelism and Infinity.
We mentioned earlier about the central and parallel projection and that we had to distinguish between them.
Well there is also the thorny issue of the parallel lines.

All lines intersect eventually apart from parallel lines. 
This hampered geometry for a long time because it is an exception case.  

Now you may have heard the statement that parallel lines meet at infinity and thought that the statement is a little unsatisfying and not useful. 

Computers for example, cannot handle infinity so it's not much good to us.
       

## Ideal Points.
To make this all easier, mathematicians sought extensions to the basic concepts that would include these exceptions in a unifying group.

So we start with the idea of points at infinity (often called ideal points). 

And with this we include the idea that parallel lines meet at infinity i.e. they meet at an ideal point. \\We can also say that a parallel projection is a central projection with the central point $0$ at infinity.


We treat ideal points just as if they were ordinary points in the plane or space.
So we want the same set of rules that determine the behaviour of points, lines and planes to persist even when these elements are ideal.

So we declare (or agree) to add to the ordinary points on each line, a single ideal point. 

This point shall be considered to belong to all the lines parallel to our line but it will belong to no other lines.

The consequences of this is that every pair of lines in the plane will now intersect in a single point. 

If the lines are not parallel they will intersect in an ordinary point. 

If the lines are parallel, they will intersect in the ideal point common to the two lines.

The ideal point on a line is also called the point at infinity on an line.
       


**Note: don't be tempted to think there are two ideal points on a line, i.e. at $\pm\infty$. 
The point at infinity is to unify the idea that two lines only meet at a single point. 
Two ideal points would mean that they meet at two points. 
Also we state that through any two points, one and only one line can be drawn. 
Two ideal points would allow a second line to be drawn for every point. 
Since these ideal points would be in common with other parallel lines effectively we allow an infinite number of lines to be drawn through the two ideal points. So forget I mentioned it.**
       

## Ideal Line

Now despite all the ideal points being at infinity this does not mean all the ideal points are the one point. 

This would break the rules. 

So instead we agree to add to the ordinary lines in the plane, a single ideal line (the line at infinity in the plane), containing all the ideal points in the plane but it does not contain any other points i.e. no ordinary points.

This is in accordance with our other rules that only a single line can pass through any two points. 
    
Therefore only an ideal line could pass through two ideal points. 

Also it cannot be an ordinary line it must be an ideal line because ordinary lines must contain only one ideal point. 

It cannot contain any ordinary points as any line that contains an ordinary point and an ideal point must be an ordinary line. 

And the ideal line must contain absolutely all of the ideal points as we want to have a point in common with every ordinary line.
       

## Horizon Vs Line at infinity

A word of caution here when you read texts on geometry that refer to the line at infinity. 
It can be easy to become convinced that there is something special about the horizon line. 

Now we don't live on a flat earth but to a rough approximation you can think of it as just that, a flat plane, and as such the horizon is the line at infinity. 

But if you were beside a very long wall that went all the way to the horizon, then that would end in a vertical line at infinity. 

We are discussing planes only here, so far, therefore don't let your 3D world confuse you. 
When we get to 3D we will talk about a plane at infinity.

## Representing a point at infinity

You cannot use the $\infty$ symbol to represent a point at infinity as there are many points at infinity so instead it is represented by a family of parallel lines.

The intersection at infinity is just a convenient way of stating that lines are parallel, which was the exception we were trying to deal with. 

These can now be dealt with just like the ordinary cases.

       

## The 3D case.
In 3D we have a similar situation but it is not as easy to get an intuitive sense of it. 

This is because it is difficult for us to draw the 3D version on a 2D page.

Nonetheless, we take the same strategy as before. 

We agree to introduce a point at infinity associated with every family of parallel lines. 

Each plane has a line at infinity. In the 2D case we introduced the line at infinity so here we have to introduce the plane at infinity. This is the plane that contains all the lines at infinity from all the possible planes. 
We say that each ordinary plane intersects the plane at infinity in its line at infinity.
       

## Duality

In projective geometry, all the theorems occur in pairs. This is referred to as duality. 

For example in 2D plane geometry, the point and the line are called dual elements. Drawing a line through a point or marking a point on a line are called dual operations. 

You can consider two figures to be dual if you can obtain one from the other by replacing each element and operation by its dual element or operation. 

This is distinct to projective geometry. Metric geometry does not have this characteristic.
       

## Analytic Representation
_"The general tendency in modern mathematics is to base everything on the number concept, and in geometry this tendency, which started with Fermat and Descartes, has had decisive triumphs. Analytic geometry has developed from the status of a mere tool in geometrical reasoning to a subject where the intuitive geometrical interpretation of the operations and results is no longer the ultimate and exclusive goal, but has rather the function of a guiding principle that aids in suggesting and understanding the analytical results."_

[Courant, Robbins; What is Mathematics?](https://www.amazon.co.uk/Mathematics-Elementary-Approach-Ideas-Methods/dp/0195105192/ref=sr_1_1?ie=UTF8&qid=1550573453&sr=8-1&keywords=what+is+mathematics+courant+and+robbins)
       

## Analytic Representation of a line

To embark on the journey of analytical geometry we must introduce numbers. 

We do this by introducing a coordinate frame. 

The two most common coordinate frames are $(x,y)$ Cartesian (Rectangular) coordinates, and $(\rho, \theta)$ Polar coordinates.

We must be able to define an object completely with these.
So a point is defined by one set of coordinates.
A line can be defined as all the points (coordinates) that satisfy a linear equation.

$$ ax  +by +c = 0$$

You can then give the coordinates of a line as the (a,b,c) values, i.e. you need three of them.
       

## Analytic Representation of a Circle

$x^2 + y^2 = r^2$ - Equation of a circle, with center at the origin $ (x=0,y=0) $ and radius $ r$.


$(x-a)^2 + (y-b)^2 = r^2$ - Equation of a circle, with center at - $(x=a,y=b)$ - and radius $r$.

$\frac{x^2}{a^2}+\frac{y^2}{b^2} = 1$ -  An ellipse

## Homogeneous Coordinates

We have mentioned these in a previous lecture and said we would come back to them. 
Homogeneous coordinates solve a number of problems.
Firstly they encode our new ideas (agreements) about incorporating ideal points (points at infinity). 

Secondly they can make an affine problem into a linear problem, which greatly eases our use of linear algebra.
This is how you saw them being used earlier.


2D Homogeneous are the easiest to intuitively understand but the 3D version is a direct extension. So if you can picture the 2D case you can accept the 3D case.

So take the points on a 2D (x,y) plane. 

Now add a third dimension z. But this does not act like the ordinary z-axis.
Move the entire x-y plane out to position 1 on the z-axis.

Now imagine that the origin (x=0,y=0,z=0) is the center of a projection. Turn every point in (x,y) into a line that passes through the origin and the point (x,y). 
Now imagine that you put another (x,y) plane at position 2 on the z-axis.
       


Question: If a line passes through the origin (x=0,y=0,z=0) and through a point (x,y,z=1) where will that line intersect the x-y plane at z=2.

Answer (2x,2y,2).

This is the case for any homogeneous coordinate, (kx,ky,k).

This is what we meant by the z-dimension not being independent of the others.
To get back to the standard coordinates you divide across by k.
       


Now what about points at infinity? 

In the standard x-y plane you would have to describe these as $(\infty, \infty)$ and there is no way to tell any ideal points apart. 

In homogeneous coordinates you do this as follows. You set the value of k to zero.
e.g. If the ideal point is given as (3,2,0), this tells us we have a point at infinity in the direction (3,2). What are it's standard coordinates? Divide across by k=0.\\ $(\frac{3}{0}, \frac{2}{0}) = (\infty, \infty)$ 

So in homogeneous coordinates we can treat them as ordinary points that have a z vaule of 0. But in standard coordinates we would have to deal with them as exceptional cases.


How do homogeneous coordinates make an affine problem linear? 
Well we turn every point in the affine space into a line that passes through the origin. 

Lines that pass through the origin are linear. 
       

## Extension to 3D

3D is not as easy to visualise as you have to create a $4^{th}$ dimension to project from but the analytical process is the exact same.

A point in (x,y,z) becomes (kx,ky,kz,k) and the easiest thing for us to do is to choose k=1 so we get (x,y,z,1).
If the point is an ideal point, k is set to 0.

Working back the way you just divide across by the $4^{th}$ dimension.

## Do we lose all the preservations?

So you may remember that rigid body transformations preserved angles and magnitudes but projective did not. 

Does this mean that moving to homogeneous coordinates loses us those rigid body preservations? 

The answer is no. 

We get away with it because while the homogeneous coordinates are a projection, they are a projection from a higher dimension.

Projective preserves straight lines. 

The straight lines are all we require to get back to the lower dimensions. 

## What is the link between homogeneous coordinates and a camera projection?

Well they are both projections but that's where it ends. Homogeneous coordinates unify ideal points and ordinary points and can make an affine transformation linear. 

A camera projection is not solving any such problem. If anything, it _is_ a problem. We would like to capture the 3D world but the camera can only capture 2D and that 2D plane that is captured is a projection.

So a camera loses us one of our three dimensions and loses us the preservations of angles and magnitudes etc.

Homogeneous coordinates don't lose anything.

       